In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
NAME = "cifakeCNN{}".format(time.strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
input_dir = "/kaggle/input/cifar-10-fake-image-dataset/cifar-10-dataset" #! Kaggle path
working_dir = "/kaggle/working" #! Kaggle path

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # """Restrict TensorFlow to only allocate 1GB of memory on the first GPU"""
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [ ]:
#! """When running locally, limits the threads to 1 (only uses one thread of one CPU core)"""
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)

In [ ]:
#! """When running on Kaggle, copies the input directory cifake_hyperband to the working directory for tuner continuity / cumulative session training."""
# from distutils.dir_util import copy_tree
# source_dir = "/kaggle/input/cifake_hyperband/"
# destination_dir = "/kaggle/working/"
# copy_tree(source_dir, destination_dir)

In [ ]:
ds_train = image_dataset_from_directory(
    f'{input_dir}/train',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
#     seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
ds_test = image_dataset_from_directory(
    f'{input_dir}/test',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
#     seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [ ]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True)


In [ ]:
backup_dir = f"{working_dir}/backup/latest"
backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir, save_freq="epoch", delete_checkpoint=True
) 

In [ ]:
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
    if epoch < 11:
        return lr
    else:
        return lr * 0.98

schedule_lr = LearningRateScheduler(scheduler)

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# timestamp = time.strftime("%Y%m%d-%H%M%S")
# model_checkpoint = ModelCheckpoint(
#     f'/kaggle/working/best/best_model_{timestamp}.keras', monitor='val_loss', save_best_only=True, mode='min'
# )

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.0008)

In [ ]:
def MBConv6(input_tensor, hp=None, out_channels=32, expansion=6, squeezing=4, use_se=True, dropout_rate=0.0):
    # inverted residual structure
    # pointwise convolution 1 expansion
    x = layers.Conv2D(expansion * input_tensor.shape[-1], (1,  1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise Separable Convolution 
    x = layers.DepthwiseConv2D(kernel_size=(3,  3), strides=(1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # pointwise convolution 2 bottleneck
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Dropout
    if dropout_rate >  0.0:
        x = layers.Dropout(dropout_rate)(x)

    # bottleneck
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape(se_shape)(se)
        se = layers.Conv2D(out_channels // squeezing, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation(swish)(se)
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation('sigmoid')(se)
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels:
        shortcut = input_tensor
    else:
        shortcut = layers.Conv2D(out_channels, (1,  1), strides=(1,  1), padding='same', use_bias=False)(input_tensor)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])

    return x
input_tensor = layers.Input(shape=(32,  32,  16)) 
output_tensor = MBConv6(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

In [ ]:

def MBConv1(input_tensor, hp=None, expansion=4, squeezing=4, out_channels=16, strides=(1,   1), use_se=True):

    x = layers.Conv2D(expansion, (1,   1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)


    x = layers.DepthwiseConv2D(kernel_size=(3,   3), strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x) 


    x = layers.Conv2D(out_channels, (1,   1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape((1,  1, out_channels))(se) 
        se = layers.Conv2D(out_channels //  squeezing, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation(swish)(se)   
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation('sigmoid')(se)
        se = layers.Reshape(se_shape[1:])(se)  # Reshape to original
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels and strides == (1,   1):
        x = layers.Add()([x, input_tensor])
    else:
        input_tensor = layers.Conv2D(out_channels, (1,   1), strides=strides, padding='same', use_bias=False)(input_tensor)
        x = layers.Add()([x, input_tensor])

    return x

input_tensor = layers.Input(shape=(32,  32,  3)) 
output_tensor = MBConv1(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

In [ ]:
import keras_tuner
from keras_tuner import RandomSearch, Hyperband
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy, Precision, Recall, F1Score

def build_model(hp=None):
    activation_function = activation=hp.Choice('ACTIVATION', values=['swish', 'gelu', 'silu'])
    
    input_tensor = layers.Input(shape=[32,  32,  3])
    x = layers.Reshape(target_shape=(32, 32, 3))(input_tensor)
    
    # x = layers.RandomRotation(factor=0.1488228133769420)(input_tensor)
    x = layers.RandomTranslation(
        height_factor=hp.Float('height_factor', min_value=0.0001, max_value=0.1, sampling='log'),
        width_factor=hp.Float('width_factor', min_value=0.0001, max_value=0.1, sampling='log')
    )(x)
    if hp.Boolean('PREPROCESSING_horizontal'):
        x = layers.RandomFlip(mode='horizontal')(x)
    # x = layers.BatchNormalization()(x)
    if hp.Boolean('PRECONV_norm'):
        x = layers.BatchNormalization()(x)

    # x = MBConv1(x, hp=hp)
    num_conv_layers = hp.Int('num_conv_layers', min_value=2, max_value=5, step=1)
    max_pool_layers = hp.Int('max_pool_layers', min_value=0, max_value=num_conv_layers-1, step=1)
    batch_norm_layers = hp.Int('batch_norm_layers', min_value=2, max_value=num_conv_layers, step=1)

    max_pool_assigned = [0] * num_conv_layers
    batch_norm_assigned = [0] * num_conv_layers

    unassigned_indices_max_pool = set(range(num_conv_layers))
    unassigned_indices_batch_norm = set(range(num_conv_layers))

    for _ in range(max_pool_layers):
        index = np.random.choice(list(unassigned_indices_max_pool))
        max_pool_assigned[index] = 1
        unassigned_indices_max_pool.remove(index)

    for _ in range(batch_norm_layers):
        index = np.random.choice(list(unassigned_indices_batch_norm))
        batch_norm_assigned[index] = 1
        unassigned_indices_batch_norm.remove(index)

    for i in range(num_conv_layers):
        x = layers.Conv2D(filters=hp.Int(f'conv_{i}_filter', min_value=42, max_value=54, step=6),
                                 kernel_size=hp.Choice(f'conv_{i}_kernel', values=[3, 4, 5]),
                                 activation=activation_function, padding='same',
                                 kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        x = layers.Dropout(hp.Float(f'CONV_dropout_{i}_rate', min_value=0.0, max_value=0.3, step=0.1))(x)

        if max_pool_assigned[i] == 1:
            x = layers.MaxPool2D()(x)
        if batch_norm_assigned[i] == 1:
            x = layers.BatchNormalization()(x)
        
    
    
    if hp.Boolean('use_maxpool_final'):
        x = layers.MaxPool2D()(x)
    if hp.Boolean('use_batchnorm_final'):
        x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)

    for i in range(hp.Int('num_dense_layers', min_value=6, max_value=12, step=1)):   
        x = layers.Dense(hp.Int(f'dense_{i}_units', min_value=444, max_value=544, step=32),
                         activation=activation_function,
                         kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        x = layers.Dropout(hp.Float(f'dropout_{i}_rate', min_value=0.0, max_value=0.6, step=0.1))(x)
    
    x = layers.Dense(1, activation='sigmoid')(x)
    
    optimizer_choice = hp.Choice('optimizer', values=['Adagrad', 'Adadelta', 'Adamax'])
    
    if optimizer_choice == 'Adamax':
        optimizer = tf.keras.optimizers.Adamax(learning_rate=hp.Float('adamax_learning_rate', min_value=0.0018, max_value=0.0098, sampling='LOG'))
    elif optimizer_choice == 'Adadelta':
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=hp.Float('adadelta_learning_rate', min_value=0.0018, max_value=0.0098, sampling='LOG'))
    elif optimizer_choice == 'Adagrad':
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=hp.Float('adagrad_learning_rate', min_value=0.0018, max_value=0.0098, sampling='LOG'))
  
    
    model = Model(inputs=input_tensor, outputs=x)

    

    model.compile(optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['binary_accuracy', 
                      Precision(name='precision'), 
                      Recall(name='recall'), 
                      F1Score(name='f1_score')])
    return model


In [ ]:
tuner = Hyperband(
    build_model,
    objective='val_binary_accuracy', 
    max_epochs=13,
    factor=3,
    directory=f'{working_dir}/projects',
    project_name='cifake_hyperband')

In [ ]:
# Start the hyperparameter search with early stopping
tuner.search(ds_train, epochs=13, validation_data=ds_valid, callbacks=[early_stopping, tensorboard, reduce_lr, schedule_lr, backup_callback]) #, verbose=0 model_checkpoint

best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save(f"{working_dir}/BEST{NAME}.keras")